# Data Preperation

### Input:
#### 1) CSV of Targets for each image
#### 2) All folders of train images 
### Output:
High level directory called data is created. Inside of high level folder are two major directories of Train and Val. Each major director has a seperate folder for each of the 5 classes
### Next Steps:
Move high level directory and all contents next to script which will train model

In [1]:
import numpy as np
import cv2

# Import Image

In [3]:
# Load an color image in color
img = cv2.imread('../SML_Project_Data/train/10_left.jpeg',1)
print(img.shape)
img

(3168, 4752, 3)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

# View Image

In [ ]:
# Used to display image in window, results the kernel to die though.
#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Create PNG of Image Object

In [6]:
cv2.imwrite('10_left_new.jpeg',img) # Confirmed manually the exported image is the correct size

True

# Data Notes
- Each eye can have a different disease value. However, there is strong correlation of the disease one eye to other eye
- Images are different sizes, but it is easy to rescale all images to a set size
- Colors have color, but not too much. Can try to convert to greyscale to reduce the bytes of the image which results in more room to have a larger image with more pixels. Note, this will require changing the model because it expects a size of (x, y, 3) of each of Red Green and Blue.
- Classes are very unballanced with most being class 0. This must be solved before upload to system which trains model.
- The Test data has no labels, so we should use the training data to evaluate the models
- Each eye from a unique person has images next to each other in the training set
- The JPEG are compressed versus RGB matrices, so they are smaller than those tensors by a factor of 15
- If we use different format of the data, such as using a sick/healthy class this is done by changing the final layer of the CNN model as well as the Keras DirectoryIterator. This can be done by making sure that there are two folders inside of the train and val subfolder.
# Code Notes
- Keras CNN will run regardless of the input image size. However, would likely be best to alter internal structure depending upon input image size
# Notes of Manual Steps
- Ballancing the classes
- Moving images into the proper folders of Google Colab
# Next Steps
See how to convert Keras code to use classification instead of binary! (Done)
Use all data with a set target upload size

# Inspect Image Object

In [7]:
img.shape

(3168, 4752, 3)

In [11]:
img[1500][2300][1]

146

In [12]:
np.max(img)

255

In [13]:
np.min(img)

0

In [24]:
print('Original Size', img.shape)

print("%d bytes in original image" % (img.size * img.itemsize))

print('MB in the original array image is', img.size * img.itemsize / 1000000, 'versus the original image JPEG has 1.5 MB') # 1 million bytes in a MB

resized = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
print('Resized Size', resized.shape)
cv2.imwrite('10_left_reshaped_new.jpeg',resized) # Goes from 1.5 MB to 3 KB which redices the size by a factor 1 K

print("%d bytes in compressed image" % (resized.size * resized.itemsize))

print('Original pixel size is', np.prod(img.shape))
print('New pixel size is', np.prod(resized.shape))
print('Factor of reduction is by', np.prod(img.shape) / np.prod(resized.shape))


Original Size (3168, 4752, 3)
45163008 bytes in original image
MB in the original array image is 45.163008 versus the original image JPEG has 1.5 MB
Resized Size (100, 100, 3)
30000 bytes in compressed image
Original pixel size is 45163008
New pixel size is 30000
Factor of reduction is by 1505.4336


# Downscale Image to 100 x 100 x 3 and Export

In [15]:
from pathlib import Path
import pandas as pd

In [20]:
train_target = pd.read_csv('../SML_Project_Data/trainLabels.csv', delimiter=',')
print(len(train_target))
train_target.head()

35126


,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [38]:
total_images = 0
for index, row in train_target.iterrows():
    split_image = row['image'].split('_')
    image_num = split_image[0]
    side = split_image[1]
    
    image_name = str(image_num) + '_' + side + '.jpeg'

    train_image_path = '../SML_Project_Data/train/' + image_name

    my_file = Path(train_image_path)

    try:
        my_abs_path = my_file.resolve(strict=True)
    except FileNotFoundError:
        pass
    else:
        total_images = total_images + 1
total_images

8408

In [37]:
counter = 0

status = int(total_images / 20)
print('Will give a status every', status, 'images')

for index, row in train_target.iterrows():
    #print(row['image'], row['level'])
    
    split_image = row['image'].split('_')
    image_num = split_image[0]
    side = split_image[1]

    image_name = str(image_num) + '_' + side + '.jpeg'

    train_image_path = '../SML_Project_Data/train/' + image_name

    my_file = Path(train_image_path)

    try:
        my_abs_path = my_file.resolve(strict=True)
    except FileNotFoundError:
        pass
    else:
        counter = counter + 1
        
        # Pull Image
        img2 = cv2.imread(train_image_path, 1)
        # Resize
        resized = cv2.resize(img2, (100, 100), interpolation = cv2.INTER_AREA)
        # Create New Image File
        cv2.imwrite('../SML_Project_Data/resized_train/resized_train_' + str(row['level']) + '/' + image_name, resized)
        resized
    if counter % status == 0:
        print('At', counter, 'images out of', total_images)
        
print('Processed Imnage Count', counter)

351
At 351 percent
At 702 percent
At 1053 percent
At 1404 percent
At 1755 percent
At 2106 percent
At 2457 percent
At 2808 percent
At 3159 percent
At 3510 percent
At 3861 percent
At 4212 percent
At 4563 percent
At 4914 percent
At 5265 percent
At 5616 percent
At 5967 percent
At 6318 percent
At 6669 percent
At 7020 percent
At 7371 percent
At 7722 percent
At 8073 percent
Processed Imnage Count 8408


In [46]:
import os

for disease_type in (0, 1, 2, 3, 4):
    path, dirs, files = next(os.walk('../SML_Project_Data/resized_train/resized_train_' + str(disease_type)))
    file_count = len(files)
    print('Disease Class of', disease_type, 'has count of images of', file_count)

Disease Class of 0 has count of images of 6150
Disease Class of 1 has count of images of 588
Disease Class of 2 has count of images of 1283
Disease Class of 3 has count of images of 221
Disease Class of 4 has count of images of 166
